In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import duckdb
import numpy as np
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('my_database.duckdb')

In [3]:
import pandas as pd
import re

# Read the CSV file line by line
file_path = 'arbitrum_stgusdc_decoded.csv'
lines = []

with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize dictionaries to store lines for each transaction type
transaction_dict = {}

# Function to parse fields without combining text between single quotes
def parse_line(line):
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    return fields

# Iterate over each line to categorize it based on the 6th field
for line in lines:
    fields = parse_line(line.strip())
    if len(fields) > 5:  # Ensure there are at least 6 fields after cleaning
        transaction_type = fields[5]
        if transaction_type not in transaction_dict:
            transaction_dict[transaction_type] = []
        transaction_dict[transaction_type].append(fields)

# Create separate DataFrames for each transaction type
dataframes = {}
for transaction_type, data in transaction_dict.items():
    # Determine the number of columns for this transaction type
    num_columns = max(len(fields) for fields in data)
    # Pad the data with empty strings to ensure all rows have the same number of columns
    padded_data = [fields + [''] * (num_columns - len(fields)) for fields in data]
    # Create DataFrame
    df = pd.DataFrame(padded_data, columns=[f'Field_{i+1}' for i in range(num_columns)])
    dataframes[transaction_type] = df

# Print out the names of the created DataFrames
for transaction_type in dataframes.keys():
    print(f'Created DataFrame: {transaction_type}')

SwapFeePercentageChanged = dataframes['SwapFeePercentageChanged']


Created DataFrame: SwapFeePercentageChanged
Created DataFrame: Transfer
Created DataFrame: Approval


In [4]:
SwapFeePercentageChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,8114577,0,0,0x31330375b2af3306733f2fddf6a7ea6e7d888fd0c94a9a42cb0f3c8bd9282c1f,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,3000000000000000,None,None,None,None,None,None,None,None,None
1,105431914,3,6,0xf5bfeb3da69a2223e3cae8da625afd71877d34ccc7e05eab3cc54555789dc94d,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,600000000000000,None,None,None,None,None,None,None,None,None
2,105431914,3,12,0xf5bfeb3da69a2223e3cae8da625afd71877d34ccc7e05eab3cc54555789dc94d,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,3000000000000000,None,None,None,None,None,None,None,None,None
3,105433562,1,3,0x277e802d8f8a8cfae2fc32650efcc80a278131e8792367f00057057229c10723,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,600000000000000,None,None,None,None,None,None,None,None,None
4,105433562,1,14,0x277e802d8f8a8cfae2fc32650efcc80a278131e8792367f00057057229c10723,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,3000000000000000,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,217297020,5,31,0x336605f51fe2b6e3194950293bd0da07659d00e9253266238c8ea28250da2ba1,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,3000000000000000,None,None,None,None,None,None,None,None,None
3711,217320778,3,4,0x0bd248db475f52e1c7d3cf9ae9123f90e50bb05dfec715d7a9cc664076d1b3c8,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,600000000000000,None,None,None,None,None,None,None,None,None
3712,217320778,3,17,0x0bd248db475f52e1c7d3cf9ae9123f90e50bb05dfec715d7a9cc664076d1b3c8,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,3000000000000000,None,None,None,None,None,None,None,None,None
3713,217377258,2,4,0x8821abcac9db864f574d8e5f3517a40117ce903e4ae525eb8753d98dffa21bbf,0x3a4c6D2404b5eb14915041e01F63200a82f4a343,SwapFeePercentageChanged,600000000000000,None,None,None,None,None,None,None,None,None


In [5]:
selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'fee'}
SwapFeePercentageChanged = SwapFeePercentageChanged[list(selected_columns.keys())].rename(columns=selected_columns)

In [6]:
SwapFeePercentageChanged

,block_number,log_index,transaction_hash,event,fee
0,8114577,0,0x31330375b2af3306733f2fddf6a7ea6e7d888fd0c94a9a42cb0f3c8bd9282c1f,SwapFeePercentageChanged,3000000000000000
1,105431914,6,0xf5bfeb3da69a2223e3cae8da625afd71877d34ccc7e05eab3cc54555789dc94d,SwapFeePercentageChanged,600000000000000
2,105431914,12,0xf5bfeb3da69a2223e3cae8da625afd71877d34ccc7e05eab3cc54555789dc94d,SwapFeePercentageChanged,3000000000000000
3,105433562,3,0x277e802d8f8a8cfae2fc32650efcc80a278131e8792367f00057057229c10723,SwapFeePercentageChanged,600000000000000
4,105433562,14,0x277e802d8f8a8cfae2fc32650efcc80a278131e8792367f00057057229c10723,SwapFeePercentageChanged,3000000000000000
...,...,...,...,...,...
3710,217297020,31,0x336605f51fe2b6e3194950293bd0da07659d00e9253266238c8ea28250da2ba1,SwapFeePercentageChanged,3000000000000000
3711,217320778,4,0x0bd248db475f52e1c7d3cf9ae9123f90e50bb05dfec715d7a9cc664076d1b3c8,SwapFeePercentageChanged,600000000000000
3712,217320778,17,0x0bd248db475f52e1c7d3cf9ae9123f90e50bb05dfec715d7a9cc664076d1b3c8,SwapFeePercentageChanged,3000000000000000
3713,217377258,4,0x8821abcac9db864f574d8e5f3517a40117ce903e4ae525eb8753d98dffa21bbf,SwapFeePercentageChanged,600000000000000


In [8]:
con.execute('CREATE TABLE arbitrum_stgusdc_fee_change AS SELECT * FROM SwapFeePercentageChanged').fetchdf()

# Commit the changes
con.commit()

# Close the connection
con.close()